In [1]:
!rm -rf predict.txt predict.zip

In [ ]:
import cv2
import numpy as np

class DayNightClassifier:
    def __init__(self,
                 night_intensity_ratio_threshold: float = 0.55,
                 brightness_threshold: int = 95):

        self.brightness_threshold = brightness_threshold
        self.night_intensity_ratio_threshold = night_intensity_ratio_threshold

    @staticmethod
    def enhance_contrast(image):
        clahe = cv2.createCLAHE(clipLimit=2.15, tileGridSize=(8, 8))
        enhanced_image = clahe.apply(image)
        return enhanced_image

    def calculate_histogram_and_brightness(self, image_path):
        # Đọc hình ảnh và chuyển sang grayscale
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            return None, None

        # Tăng cường độ tương phản
        image = self.enhance_contrast(image)

        # Tính toán histogram
        hist = cv2.calcHist([image], [0], None, [256], [0, 256])
        hist = hist / hist.sum()

        # Tính độ sáng trung bình
        avg_brightness = np.mean(image)

        # Tính tỷ lệ pixel tối
        dark_pixel_ratio = hist[:self.brightness_threshold].sum()

        return dark_pixel_ratio, avg_brightness

    def isDay(self, image_path):
        dark_pixel_ratio, avg_brightness = self.calculate_histogram_and_brightness(image_path)

        if dark_pixel_ratio is None or avg_brightness is None:
            return None

        # Phân loại dựa trên tỷ lệ pixel tối
        if dark_pixel_ratio > self.night_intensity_ratio_threshold:
            return False  # Đêm

        # Phân loại dựa trên độ sáng trung bình
        return avg_brightness > self.brightness_threshold  # Ngày nếu độ sáng cao hơn ngưỡng

# Sử dụng method isDay để kiểm tra
classifier = DayNightClassifier(night_intensity_ratio_threshold=0.54, brightness_threshold=93)

In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

In [ ]:
from IPython.display import clear_output
from tqdm import tqdm
import os
import glob
from ultralytics import YOLO

# Đọc danh sách tệp ảnh
public_test = glob.glob("../data/raw/public test/*.jpg")
batch_size = 64  # Đặt số lượng ảnh trong mỗi batch, điều chỉnh sao cho phù hợp với RAM của bạn

model_day = YOLO('../models/base/day/day.pt', verbose=False)
model_night = YOLO('../models/base/night/night.pt', verbose=False)

# Hàm chuyển đổi tọa độ từ xyxy sang xywh
def xyxy_to_xywh(xyxy, img_width, img_height):
    xmin, ymin, xmax, ymax = xyxy
    x_center = (xmin + xmax) / 2 / img_width
    y_center = (ymin + ymax) / 2 / img_height
    width = (xmax - xmin) / img_width
    height = (ymax - ymin) / img_height
    return x_center, y_center, width, height

# Xử lý theo batch và ghi kết quả
with open("predict.txt", "w") as f:
    for i in tqdm(range(0, len(public_test), batch_size)):
        clear_output()
        batch_files = public_test[i:i + batch_size]
        results = []  # Danh sách để lưu trữ kết quả từ cả hai mô hình
        # Phân loại ngày/đêm và dự đoán với mô hình tương ứng
        for img_path in batch_files:
            if classifier.isDay(img_path):
                # print(img_path)
                # print("day")
                results.extend(model_day([img_path])) # Dự đoán với model1 cho ban ngày
            else:
                # print(img_path)
                # print("night")
                results.extend(model_night([img_path]))  # Dự đoán với model2 cho ban đêm
        # from IPython.display import clear_output
        # clear_output()

        for result in results:
            img_path = result.path
            img_name = os.path.basename(img_path)
            img_width, img_height = result.orig_img.shape[1], result.orig_img.shape[0]

            for box in result.boxes:
                xyxy = box.xyxy[0].cpu().numpy()
                class_id = int(box.cls[0].item())
                confidence = box.conf[0].item()
                x_center, y_center, width, height = xyxy_to_xywh(xyxy, img_width, img_height)

                # Ghi kết quả vào tệp
                # f.write(f"{img_name} {class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f} {confidence:.6f}\n")
                f.write(f"{img_name} {class_id} {x_center} {y_center} {width} {height}\n")


0: 384x640 5 motorbikes, 1 car, 1 truck, 17.3ms
Speed: 1.7ms preprocess, 17.3ms inference, 5.1ms postprocess per image at shape (1, 3, 384, 640)


100%|██████████| 2060/2060 [03:13<00:00, 10.66it/s]


In [3]:
!zip -r predict.zip predict.txt

  adding: predict.txt (deflated 74%)
